In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 256 kB in 3s (102 kB/s)
Reading p

In [2]:
# postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-05 05:11:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  6.23MB/s    in 0.2s    

2022-09-05 05:11:22 (6.23 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
## add session + driver to spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# extract 
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://bootcamp-adri.s3.us-west-1.amazonaws.com/data/BootCampProject/2015_Survey.csv"
spark.sparkContext.addFile(url)
survey_data_df = spark.read.csv(SparkFiles.get("2015_Survey.csv"), sep=",", header=True, inferSchema=True)

In [5]:
#df 
# Show DataFrame
survey_data_df.show(10)

+------+------+-----------+------+-----+-------+--------+----------+----------+--------+--------+--------+--------+--------+------+--------+-------+--------+--------+--------+------+--------+--------+------+--------+-------+-------+--------+--------+--------+--------+--------+-------+--------+-------+------+--------+-------+-------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+---+-------+-----+--------+--------+--------+-------+--------+-------+--------+-------+--------+-------+-------+--------+--------+--------+-----+------+--------+--------+--------+--------+--------+--------+--------+-------+-------+--------+--------+--------+--------+------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+-------+--------+--------+--------+--------+---

In [6]:
#select relevant columns 
survey_data_df = survey_data_df.select(['_STATE','FMONTH','IDATE','IMONTH','IDAY','IYEAR','DISPCODE','SEQNO','PHYSHLTH','POORHLTH','GENHLTH','MENTHLTH','BPHIGH4','TOLDHI2','CVDINFR4','CVDCRHD4','CVDSTRK3','CHCSCNCR','CHCOCNCR','CHCCOPD1','HAVARTH3','CHCKIDNY','DIABETE3','SEX','MARITAL','EDUCA','RENTHOM1','VETERAN3','EMPLOY1','CHILDREN','INCOME2','INTERNET','QLACTLM2','USEEQUIP','BLIND','DECIDE','DIFFWALK','DIFFDRES','DIFFALON','EXRACT11','PREDIAB1','INSULIN','PAINACT2','QLMENTL2','QLSTRES2','QLHLTH2','SCNTMNY1','SXORIENT','TRNSGNDR','EMTSUPRT','LSATISFY','MSCODE','_RFHLTH','_HCVU651','_RFHYPE5','_CHOLCHK','_RFCHOL','_MICHD','_LTASTH1','_ASTHMS1','_DRDXAR1','_PRACE1','_AGEG5YR','_AGE65YR','HTIN4','_BMI5','_BMI5CAT','_CHLDCNT','_EDUCAG','_INCOMG','_DRNKWEK','_RFDRHV5','_FRTLT1','_VEGLT1','PA1MIN_','PA1VIGM_','_PACAT1','_PAINDX1','_PAREC1'])
survey_data_df.show(10)



+------+------+-----------+------+-----+-------+--------+----------+--------+--------+-------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+-------+-----+--------+--------+-------+--------+-------+--------+--------+--------+-----+------+--------+--------+--------+--------+--------+-------+--------+--------+--------+-------+--------+--------+--------+--------+--------+------+-------+--------+--------+--------+-------+------+--------+--------+--------+-------+--------+--------+-----+-----+--------+--------+-------+-------+--------+--------+-------+-------+-------+--------+-------+--------+-------+
|_STATE|FMONTH|      IDATE|IMONTH| IDAY|  IYEAR|DISPCODE|     SEQNO|PHYSHLTH|POORHLTH|GENHLTH|MENTHLTH|BPHIGH4|TOLDHI2|CVDINFR4|CVDCRHD4|CVDSTRK3|CHCSCNCR|CHCOCNCR|CHCCOPD1|HAVARTH3|CHCKIDNY|DIABETE3|SEX|MARITAL|EDUCA|RENTHOM1|VETERAN3|EMPLOY1|CHILDREN|INCOME2|INTERNET|QLACTLM2|USEEQUIP|BLIND|DECIDE|DIFFWALK|DIFFDRES|DIFFALON|EXRAC

In [7]:

#rename columns w function --> name list / or zip dictionary 

from functools import reduce
oldColumns = survey_data_df.schema.names
newColumns = ['STATE','FMONTH','DATE','MONTH','DAY','YEAR','DISPCODE','SEQNO','PHYSHLTH','POORHLTH','GENHLTH','MENTHLTH','BPHIGH4','TOLDHI2','CVDINFR4','CVDCRHD4','CVDSTRK3','CHCSCNCR','CHCOCNCR','CHCCOPD1','HAVARTH3','CHCKIDNY','DIABETE3','SEX','MARITAL_STATUS','EDUCATION','HOME_STATUS','VETERAN','EMPLOYMENT_SITUATION','CHILDREN','INCOME','INTERNET','DISABILITIES?','USEEQUIP','BLIND','ISSUES_DECISION_MAING','DIFFWALK','DIFFDRES','DIFFALON','TYPE_EXERCISE','PRE_DIABETES','INSULIN','PAINACT2','QLMENTL2','QLSTRES2','QLHLTH2','STRESSED_FOR_RENT','SXORIENT','TRNSGNDR','HAS_EMOTIONAL_SUPPORT','SATISFIED','METRO_CODE','HEALTHY_ADULTS','HEALTH_CVRAGE','BLOOD_PRESSURE','_CHOLCHK','HIGH_CHOLESTEROL','CORONARY_DISEASE','ASTHMA_DISEASE','_ASTHMS1','HAS_ARTHRITIS','RACE','AGE_14LEVEL','AGE_2LEVEL','HEIGHT','BMI','BMI_5CAT','CHILDREN2','EDUCATION_LEVEL','INCOME2','DRINKS_WEEK','HEAVY_DRINKER','FRUIT_CONSUMPTION','VEGGIE_COSUMPTION','PHYS_ACTIVITY_MINS','PA1VIGM_','PHYS_CATEGORIES','PHYS_ACT_INDX','AER_STRGTH']
survey_data_df = reduce(lambda survey_data_df, idx: survey_data_df.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), survey_data_df)
#df.printSchema()
survey_data_df.show(5)


+-----+------+-----------+-----+-----+-------+--------+----------+--------+--------+-------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+--------------+---------+-----------+-------+--------------------+--------+------+--------+-------------+--------+-----+---------------------+--------+--------+--------+-------------+------------+-------+--------+--------+--------+-------+-----------------+--------+--------+---------------------+---------+----------+--------------+-------------+--------------+--------+----------------+----------------+--------------+--------+-------------+----+-----------+----------+------+----+--------+---------+---------------+-------+-----------+-------------+-----------------+-----------------+------------------+--------+---------------+-------------+----------+
|STATE|FMONTH|       DATE|MONTH|  DAY|   YEAR|DISPCODE|     SEQNO|PHYSHLTH|POORHLTH|GENHLTH|MENTHLTH|BPHIGH4|TOLDHI2|CVDINFR4|CVDCRHD4|CVDST

In [8]:
# load in a sql function to get columns
from pyspark.sql.functions import col
#Keep Only completed Interviews
survey_data_df = survey_data_df.filter(col("DISPCODE")== 1100)

In [9]:
#ML data
ml_data_df=survey_data_df.select(['GENHLTH','MENTHLTH','PHYSHLTH','SEX','MARITAL_STATUS','EDUCATION','HOME_STATUS','VETERAN','EMPLOYMENT_SITUATION','INTERNET','DISABILITIES?','USEEQUIP','BLIND','ISSUES_DECISION_MAING','DIFFWALK','DIFFDRES','DIFFALON','HEALTHY_ADULTS','HEALTH_CVRAGE','DIABETE3','BLOOD_PRESSURE','_CHOLCHK','HIGH_CHOLESTEROL','CORONARY_DISEASE','ASTHMA_DISEASE','_ASTHMS1','HAS_ARTHRITIS','RACE','AGE_14LEVEL','BMI','BMI_5CAT','CHILDREN2','EDUCATION_LEVEL','INCOME2','DRINKS_WEEK','HEAVY_DRINKER','FRUIT_CONSUMPTION','VEGGIE_COSUMPTION','PHYS_ACTIVITY_MINS','PHYS_CATEGORIES','PHYS_ACT_INDX','AER_STRGTH'])
ml_data_df.show(5)

+-------+--------+--------+---+--------------+---------+-----------+-------+--------------------+--------+-------------+--------+-----+---------------------+--------+--------+--------+--------------+-------------+--------+--------------+--------+----------------+----------------+--------------+--------+-------------+----+-----------+----+--------+---------+---------------+-------+-----------+-------------+-----------------+-----------------+------------------+---------------+-------------+----------+
|GENHLTH|MENTHLTH|PHYSHLTH|SEX|MARITAL_STATUS|EDUCATION|HOME_STATUS|VETERAN|EMPLOYMENT_SITUATION|INTERNET|DISABILITIES?|USEEQUIP|BLIND|ISSUES_DECISION_MAING|DIFFWALK|DIFFDRES|DIFFALON|HEALTHY_ADULTS|HEALTH_CVRAGE|DIABETE3|BLOOD_PRESSURE|_CHOLCHK|HIGH_CHOLESTEROL|CORONARY_DISEASE|ASTHMA_DISEASE|_ASTHMS1|HAS_ARTHRITIS|RACE|AGE_14LEVEL| BMI|BMI_5CAT|CHILDREN2|EDUCATION_LEVEL|INCOME2|DRINKS_WEEK|HEAVY_DRINKER|FRUIT_CONSUMPTION|VEGGIE_COSUMPTION|PHYS_ACTIVITY_MINS|PHYS_CATEGORIES|PHYS_ACT_INDX|

In [10]:
 #Fill Na 
#clean byte data

ml_data_df=ml_data_df.dropna()

#remove na
ml_data_df.describe().show()


+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+-------------------+---------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|           GENHLTH|          MENTHLTH|          PHYSHLTH|                SEX|    MARITAL_STATUS|         EDUCATION|       HOME_STATUS|           VETERAN|EMPLOYMENT_SITUATION|  

In [11]:
#Replace 77/9/99 for 999
ml_data_df_cleaned=ml_data_df.na.replace([77, 9,99900,5.4E-79], [99,99,99,0])
ml_data_df_cleaned.show(10)

+-------+--------+--------+---+--------------+---------+-----------+-------+--------------------+--------+-------------+--------+-----+---------------------+--------+--------+--------+--------------+-------------+--------+--------------+--------+----------------+----------------+--------------+--------+-------------+----+-----------+----+--------+---------+---------------+-------+-----------+-------------+-----------------+-----------------+------------------+---------------+-------------+----------+
|GENHLTH|MENTHLTH|PHYSHLTH|SEX|MARITAL_STATUS|EDUCATION|HOME_STATUS|VETERAN|EMPLOYMENT_SITUATION|INTERNET|DISABILITIES?|USEEQUIP|BLIND|ISSUES_DECISION_MAING|DIFFWALK|DIFFDRES|DIFFALON|HEALTHY_ADULTS|HEALTH_CVRAGE|DIABETE3|BLOOD_PRESSURE|_CHOLCHK|HIGH_CHOLESTEROL|CORONARY_DISEASE|ASTHMA_DISEASE|_ASTHMS1|HAS_ARTHRITIS|RACE|AGE_14LEVEL| BMI|BMI_5CAT|CHILDREN2|EDUCATION_LEVEL|INCOME2|DRINKS_WEEK|HEAVY_DRINKER|FRUIT_CONSUMPTION|VEGGIE_COSUMPTION|PHYS_ACTIVITY_MINS|PHYS_CATEGORIES|PHYS_ACT_INDX|

In [12]:
#LOAD
# Store environmental variable
from getpass import getpass
password = getpass('password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://bootcampproject.cs8v5ggqsbn0.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

password··········


In [13]:
#Write DataFrame to active_user table in RDS
ml_data_df_cleaned.write.jdbc(url=jdbc_url, table='ml_data', mode=mode, properties=config)

In [14]:
#select relevant columns for dashboard
dash_data_df = survey_data_df.select(['SEQNO','STATE','FMONTH','DATE','MONTH','DAY','YEAR','METRO_CODE','RACE','AGE_14LEVEL','AGE_2LEVEL','GENHLTH','MENTHLTH','PHYSHLTH','SEX','MARITAL_STATUS','EDUCATION','HOME_STATUS','VETERAN','EMPLOYMENT_SITUATION','INTERNET','DISABILITIES?','USEEQUIP','BLIND','ISSUES_DECISION_MAING','DIFFWALK','DIFFDRES','DIFFALON','HEALTHY_ADULTS','HEALTH_CVRAGE','DIABETE3','BLOOD_PRESSURE','_CHOLCHK','HIGH_CHOLESTEROL','CORONARY_DISEASE','ASTHMA_DISEASE','_ASTHMS1','HAS_ARTHRITIS','RACE','AGE_14LEVEL','BMI','BMI_5CAT','CHILDREN2','EDUCATION_LEVEL','INCOME2','DRINKS_WEEK','HEAVY_DRINKER','FRUIT_CONSUMPTION','VEGGIE_COSUMPTION','PHYS_ACTIVITY_MINS','PHYS_CATEGORIES','PHYS_ACT_INDX','AER_STRGTH'])
dash_data_df.show(10)

+----------+-----+------+-----------+-----+-----+-------+----------+----+-----------+----------+-------+--------+--------+---+--------------+---------+-----------+-------+--------------------+--------+-------------+--------+-----+---------------------+--------+--------+--------+--------------+-------------+--------+--------------+--------+----------------+----------------+--------------+--------+-------------+----+-----------+----+--------+---------+---------------+-------+-----------+-------------+-----------------+-----------------+------------------+---------------+-------------+----------+
|     SEQNO|STATE|FMONTH|       DATE|MONTH|  DAY|   YEAR|METRO_CODE|RACE|AGE_14LEVEL|AGE_2LEVEL|GENHLTH|MENTHLTH|PHYSHLTH|SEX|MARITAL_STATUS|EDUCATION|HOME_STATUS|VETERAN|EMPLOYMENT_SITUATION|INTERNET|DISABILITIES?|USEEQUIP|BLIND|ISSUES_DECISION_MAING|DIFFWALK|DIFFDRES|DIFFALON|HEALTHY_ADULTS|HEALTH_CVRAGE|DIABETE3|BLOOD_PRESSURE|_CHOLCHK|HIGH_CHOLESTEROL|CORONARY_DISEASE|ASTHMA_DISEASE|_ASTHMS1|

In [15]:
# dashboard data

# load in a sql function to get columns
from pyspark.sql.functions import col
#Keep Only completed Interviews
clean_dash_df = dash_data_df.filter(col("DISPCODE")== 1100)

clean_dash_df=clean_dash_df.dropna()

clean_dash_df.describe().show()




+-------+--------------------+------------------+-----------------+-----------+------+------+-------+-----------------+-----------------+-----------------+-------------------+------------------+-----------------+------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+-------------------+------------------+---------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|            

In [16]:
#Clean Data replace values 

#Replace 77/9/99 for 999
dashboard_data_df =clean_dash_df.na.replace([77, 9,99900,5.4E-79], [99,99,99,0])
dashboard_data_df.show(10)



+----------+-----+------+-----------+-----+-----+-------+----------+----+-----------+----------+-------+--------+--------+---+--------------+---------+-----------+-------+--------------------+--------+-------------+--------+-----+---------------------+--------+--------+--------+--------------+-------------+--------+--------------+--------+----------------+----------------+--------------+--------+-------------+----+-----------+----+--------+---------+---------------+-------+-----------+-------------+-----------------+-----------------+------------------+---------------+-------------+----------+
|     SEQNO|STATE|FMONTH|       DATE|MONTH|  DAY|   YEAR|METRO_CODE|RACE|AGE_14LEVEL|AGE_2LEVEL|GENHLTH|MENTHLTH|PHYSHLTH|SEX|MARITAL_STATUS|EDUCATION|HOME_STATUS|VETERAN|EMPLOYMENT_SITUATION|INTERNET|DISABILITIES?|USEEQUIP|BLIND|ISSUES_DECISION_MAING|DIFFWALK|DIFFDRES|DIFFALON|HEALTHY_ADULTS|HEALTH_CVRAGE|DIABETE3|BLOOD_PRESSURE|_CHOLCHK|HIGH_CHOLESTEROL|CORONARY_DISEASE|ASTHMA_DISEASE|_ASTHMS1|

In [17]:
#Replace values from Dictionary
#stateDic={'CA':'California','NY':'New York','DE':'Delaware'}

genhlthDic={1:'Excellent',2:'Very Good',3:'Good',4:'Fair',5:'Poor',99:'Dont Know',7:'Dont know'}
YNDic={1:"Yes",2:"No",7:'Dont know',99:'Dont know'}
NYDic={1:'No',2:'Yes',7:'Dont Know',99:'Dont know'}
sexDic={1:'Male',2:'Female',7:'DOnt know',99:'Dont know'}
MetroCodeDic={1:'Center of MSA',2:'Outside Center MSA',3:'Inside a Suburban county',5:'Not in MSA', 99:'Dont know',7:'Dont know'}
RaceDic={1:'White',2:'African American',3:'American Indian',4:'Asian',5:'Pacific Islander',6:'Other',7:'No Preferred Race',8:'Multiracial',99:'Dont know'}
MaritStatDic={1:'Married',2:'Divorced',3:'Widowed',4:'Separated',5:'Never Married',6:'Member of an unmarried couple',99:'Dont know',7:'DOnt know'}
Age14Dic={1:'18-24',2:'25-29',3:'30-34',4:'35-39',5:'40-44',6:'45-49',7:'50-54',8:'55-59',9:'60-64',10:'65-69',11:'70-74',12:'75-79',13:'80-Older',14:'Don’t Know',99:'Dont know'}
Age2Dic={1:'18-64',2:'65-Older',3:'Don’t know ',7:'DOnt know'}
EduDic={1:'Did not Graduate HighSchool',2:'Graduated HighSchool',3:'Attended College',4:'Graduated College ',99:'Don’t know',7:'DOnt know'}
HomeDic={1:'Own',2:'Rent',3:'Other Arrangement',99:'Don’t Know',7:'DOnt know'}
EmplDic={1:'Employeed for wages',2:'Self-Employed',3:'Out of work>1 Year',4:'Out of work< 1 Year',5:'Homemaker',6:'Student',7:'Retired',8:'Unable to Work',99:'Don’t Know'}
IncomeDic={1:'Less than 10K',2:'10K-15K',3:'15K-20K',4:'20K-25K',5:'25K-35K',6:'35K-50K',7:'50K-75K',8:'75K or more',99:'Dont know',7:'DOnt know'}
RFDic={1:'Good or Better Health',2:'Fair or Poor Health',99:'Don’t know',7:'DOnt know'}
BMIDic={1:'Underweight',2:'Normal Weight',3:'Overweight',4:'Obese',99:'Don’t know',7:'DOnt know'}
PhysActDic={1:'Highly Active',2:'Active',3:'insufficiently Active',4:'Inactive',99:'Don’t know',7:'DOnt know'}

df2=dashboard_data_df.rdd.map(lambda x: 
    (x.SEQNO,x.STATE,genhlthDic[x.GENHLTH],x.MENTHLTH,sexDic[x.SEX],x.PHYSHLTH,MaritStatDic[x.MARITAL_STATUS],RaceDic[x.RACE],MetroCodeDic[x.METRO_CODE],Age14Dic[x.AGE_14LEVEL],Age2Dic[x.AGE_2LEVEL],EduDic[x.EDUCATION_LEVEL],HomeDic[x.HOME_STATUS],EmplDic[x.EMPLOYMENT_SITUATION],IncomeDic[x.INCOME2],RFDic[x.HEALTHY_ADULTS],BMIDic[x.BMI_5CAT],PhysActDic[x.PHYS_ACT_INDX],x.DRINKS_WEEK,YNDic[x.HAS_ARTHRITIS],YNDic[x.ASTHMA_DISEASE],NYDic[x.CORONARY_DISEASE],NYDic[x.HIGH_CHOLESTEROL],NYDic[x.BLOOD_PRESSURE],YNDic[x.HEALTH_CVRAGE],YNDic[x.INTERNET],YNDic[x.VETERAN])
    ).toDF(["SEQNO","STATE","GENHLTH","MENTHLTH","SEX","PHYSHLTH","MARITAL_STATUS","RACE","METRO_CODE","AGE_14LEVEL","AGE_2LEVEL","EDUCATION_LEVEL","HOME_STATUS","EMPLOYMENT_SITUATION","INCOME2","HEALTHY_ADULTS","BMI_5CAT","PHYS_ACT_INDX","DRINKS_WEEK","HAS_ARTHRITIS","ASTHMA_DISEASE","CORONARY_DISEASE","HIGH_CHOLESTEROL","BLOOD_PRESSURE","HEALTH_CVRAGE","INTERNET","VETERAN"])
df2.show(50)

+----------+-----+---------+--------+------+--------+--------------+----------------+--------------------+-----------+----------+--------------------+-----------------+--------------------+-------------+--------------------+-------------+-------------+-----------+-------------+--------------+----------------+----------------+--------------+-------------+--------+-------+
|     SEQNO|STATE|  GENHLTH|MENTHLTH|   SEX|PHYSHLTH|MARITAL_STATUS|            RACE|          METRO_CODE|AGE_14LEVEL|AGE_2LEVEL|     EDUCATION_LEVEL|      HOME_STATUS|EMPLOYMENT_SITUATION|      INCOME2|      HEALTHY_ADULTS|     BMI_5CAT|PHYS_ACT_INDX|DRINKS_WEEK|HAS_ARTHRITIS|ASTHMA_DISEASE|CORONARY_DISEASE|HIGH_CHOLESTEROL|BLOOD_PRESSURE|HEALTH_CVRAGE|INTERNET|VETERAN|
+----------+-----+---------+--------+------+--------+--------------+----------------+--------------------+-----------+----------+--------------------+-----------------+--------------------+-------------+--------------------+-------------+-------------+

In [18]:
#LOAD
# Store environmental variable
from getpass import getpass
password = getpass('password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://bootcampproject.cs8v5ggqsbn0.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

password··········


In [19]:
#Write Dash DF to active_user table in RDS
#df2.describe()

df2.write.jdbc(url=jdbc_url, table='dashboard_data', mode=mode, properties=config)

In [ ]:
#oldColumns = dashboard_data_df.schema.names

#oldColumns